<a href="https://colab.research.google.com/github/VellummyilumVinoth/Train_And_Test_Using_ALBERT/blob/main/FinetunedAlbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
import os
import re
from transformers import AlbertTokenizer, AlbertForMaskedLM, AdamW, Trainer, TrainingArguments, DataCollatorForLanguageModeling

import random

class VariableNamesDataset(torch.utils.data.Dataset):
    def __init__(self, variable_names, source_statements, tokenizer, mask_probability=0.8):
        self.variable_names = variable_names
        self.source_statements = source_statements
        self.tokenizer = tokenizer
        self.mask_probability = mask_probability

    def __len__(self):
        return len(self.source_statements)

    def __getitem__(self, idx):
        variable_name = str(self.variable_names[idx])
        source_statement = str(self.source_statements[idx])

        # Tokenize the variable name and source statement
        variable_name_tokens = self.tokenizer.tokenize(variable_name)
        source_statement_tokens = self.tokenizer.tokenize(source_statement)

        # Randomly select a percentage of variable name tokens to mask
        variable_name_indices = [i for i, token in enumerate(source_statement_tokens) if token in variable_name_tokens]

        num_to_mask = int(len(variable_name_indices) * (1 - self.mask_probability))
        indices_to_mask = random.sample(variable_name_indices, num_to_mask)

        # Replace the selected variable name tokens with the [MASK] token
        for i in indices_to_mask:
            source_statement_tokens[i] = '[MASK]'

        # Convert the tokens back to text
        masked_source_statement = self.tokenizer.convert_tokens_to_string(source_statement_tokens)

        # Tokenize the masked source statement
        input_ids = self.tokenizer.encode(
            masked_source_statement,
            add_special_tokens=True,
            truncation=True,
            padding='max_length',
            max_length=128
        )

        return torch.tensor(input_ids)

# Load the dataset
df = pd.read_csv('output.csv', names=['labels', 'input_ids'])

# Load the ALBERT tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForMaskedLM.from_pretrained('albert-base-v2')

# Extract variable names and source statements
variable_names = df['labels'].tolist()
source_statements = df['input_ids'].tolist()

# Split the data into train and test sets
train_size = int(len(source_statements) * 0.8)
train_variable_names = variable_names[:train_size]
train_source_statements = source_statements[:train_size]
test_variable_names = variable_names[train_size:]
test_source_statements = source_statements[train_size:]

# Prepare the data for training and testing
train_dataset = VariableNamesDataset(train_variable_names, train_source_statements, tokenizer, mask_probability=0.8)
test_dataset = VariableNamesDataset(test_variable_names, test_source_statements, tokenizer, mask_probability=0.8)

# Prepare the data for training
dataset = VariableNamesDataset(variable_names, source_statements, tokenizer)


In [3]:
# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.8
)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",   # Evaluation and Save happens every X steps
    eval_steps = 300,                # Evaluation happens every X steps
    save_steps = 500,               # Save checkpoint every X steps
    num_train_epochs = 10,            # Total number of training epochs
    learning_rate = 5e-5,            # Learning rate
    per_device_train_batch_size=32,  # Batch size per device during training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps = 500,              # Number of warmup steps
    weight_decay = 0.01,             # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=1000,              # Log every X steps
)

# Instantiate the Trainer class and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

trainer.train()

# Save the trained model and tokenizer
output_dir = './finetuned_albert'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5448
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1710
  Number of trainable parameters = 11221680


Step,Training Loss,Validation Loss
300,No log,2.312467
600,No log,1.938664
900,No log,1.857996
1200,3.100400,1.784837
1500,3.100400,1.685783


***** Running Evaluation *****
  Num examples = 1363
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1363
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1363
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1363
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1363
  Batch size = 64
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved

('./finetuned_albert/tokenizer_config.json',
 './finetuned_albert/special_tokens_map.json',
 './finetuned_albert/spiece.model',
 './finetuned_albert/added_tokens.json')

In [8]:
import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM

# Load the trained model and tokenizer
output_dir = './finetuned_albert'
model = AlbertForMaskedLM.from_pretrained(output_dir)
tokenizer = AlbertTokenizer.from_pretrained(output_dir)

# Define a sample masked statement
masked_statement = "int [MASK] = getCount();"

# Tokenize the masked statement
input_ids = tokenizer.encode(masked_statement, add_special_tokens=True, return_tensors='pt')

# Find the position of the masked token
masked_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].item()

# Generate predictions for the masked token
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Get the top 5 predictions and their probability scores
probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
top_k = torch.topk(probs, k=5)

# Print the top 5 predictions and their probability scores
for i, idx in enumerate(top_k.indices):
    token = tokenizer.convert_ids_to_tokens([idx])[0]
    print(f"Prediction {i+1}: {token}, Probability: {top_k.values[i].item()}")


loading configuration file ./finetuned_albert/config.json
Model config AlbertConfig {
  "_name_or_path": "albert-base-v2",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "vocab_size": 30000
}

loading weights file ./finetuned_albert/pytorch_model.bin
All model checkpoint weights we

Prediction 1: ", Probability: 0.2001161128282547
Prediction 2: (, Probability: 0.1409575343132019
Prediction 3: cons, Probability: 0.1175871193408966
Prediction 4: ., Probability: 0.1110304594039917
Prediction 5: t, Probability: 0.08294475078582764
